In [138]:
#init libraries + data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

#data = pd.read_csv('sample_data/mnist_train_small.csv')
data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [139]:
#inits weights and biases + activation functions
w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10)
#w_1 = np.zeros((10,784))
#w_2 = np.zeros((10,10))
#b_1 = np.zeros(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return x > 0

def sigmoid(x):
    return 1/(1 + (math.e)**(-x))

def dsigmoid(x):
    return sigmoid(x) * (1-sigmoid(x))

In [140]:
def feedforward(activation, w, b, a):
  z_out = np.matmul(w,a)
  z_out += b.reshape(b.size)
  n = z_out.size
  a_out = np.copy(z_out)
  if activation == "sigmoid":
    for i in range(n):
      a_out[i] = sigmoid(z_out[i])
  elif activation == "relu":
    for i in range(n):
      a_out[i] = relu(z_out[i])
  elif activation == "none":
    return z_out,a_out
  else:
    return "Unrecognized activation function"
  return z_out,a_out
      
def cost(a, i):
  temp = np.zeros(10)
  temp[int(Y_train[i])] = 1
  return np.sum((a-temp)**2)
  
def backprop(activation,z,a,w,dz):
  dw = np.matmul(dz.reshape((dz.size,1)),a.reshape((1,a.size)))
  #print(dw)
  dz_out = np.matmul(dz.reshape((1,dz.size)),w)
  n = dz_out.size
  if activation == "sigmoid":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * dsigmoid(z[x])
  elif activation == "relu":
    for x in range(n):
      dz_out[0][x] = dz_out[0][x] * drelu(z[x])
  else:
    return dz_out.reshape(n),dw
  return dz_out.reshape(n),dw
  
feedforward("relu",np.ones((10,10)),np.zeros(10),feedforward("relu",np.ones((10,784)),np.zeros(10),-np.ones(784))[1])

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [141]:
def relu_epoch(alpha,w_1, w_2, b_1):
  dz_2 = np.ones(10)
  for i in range(2):
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,i])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1) 
    #solution array
    y_2 = np.zeros(10) 
    y_2[int(Y_train[i])] = 1
    #dw_2
    da_2 = a_2-y_2
    for x in range(10):
      dz_2[x] = da_2[x] * drelu(z_2[x])
    dz_1, dw_2 = backprop("relu",z_1,a_1,w_2,dz_2)

    dw_1 = backprop("none",X_train[:,i],X_train[:,i],w_1,dz_1)[1]
    print(dw_2)
    w_1 -= alpha * dw_1
    w_2 -= alpha * dw_2
    b_1 -= alpha * dz_1 
    #print(w_1,w_2,b_1)
  return w_1,w_2,z_1

In [142]:
#print(np.matmul(w_1,X_train[:,1]))
#print(feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1])
#print(feedforward("relu",w_2,np.zeros(10),feedforward("relu",w_1,np.zeros(10),X_train[:,1])[1]))
for i in range(1):
    w_1, w_2, b_1 = relu_epoch(0.000001, w_1, w_2, b_1)
    z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,1])
    z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1)
    print(b_1)
    #print(cost(a_2,2))

[[8.70090947e+08 9.33226109e+08 8.67446307e+08 8.62402594e+08
  8.60732552e+08 8.82598577e+08 9.50616443e+08 7.92086724e+08
  8.24339126e+08 8.81675717e+08]
 [1.11481211e+09 1.19570462e+09 1.11142364e+09 1.10496133e+09
  1.10282158e+09 1.13083762e+09 1.21798615e+09 1.01486847e+09
  1.05619216e+09 1.12965520e+09]
 [7.05416497e+08 7.56602624e+08 7.03272386e+08 6.99183252e+08
  6.97829284e+08 7.15556919e+08 7.70701643e+08 6.42175447e+08
  6.68323721e+08 7.14808721e+08]
 [9.11398275e+08 9.77530762e+08 9.08628082e+08 9.03344920e+08
  9.01595593e+08 9.24499701e+08 9.95746697e+08 8.29690823e+08
  8.63474399e+08 9.23533029e+08]
 [8.45320556e+08 9.06658340e+08 8.42751206e+08 8.37851081e+08
  8.36228583e+08 8.57472110e+08 9.23553593e+08 7.69537015e+08
  8.00871232e+08 8.56575523e+08]
 [8.39503076e+08 9.00418735e+08 8.36951408e+08 8.32085006e+08
  8.30473674e+08 8.51571003e+08 9.17197715e+08 7.64241076e+08
  7.95359651e+08 8.50680586e+08]
 [9.45498330e+08 1.01410517e+09 9.42624490e+08 9.37143658e

In [143]:
#accuracy
def accuracy(w_1,w_2,b_1):
    n = 0
    for i in range(18999):
        a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
        a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
        max = 0
        index = 0
        for x in range(10):
            if a_2[x] > max:
                max = a_2[x]
                index = x
        if Y_train[i] == x:
            n += 1
    return n / 18999
for i in range(10):
    a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
    a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
    print(a_2,Y_train[i])

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 1
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 3
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 3
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 8
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 9
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 2
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 5
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 6
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 8
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5] 8


/tmp/ipykernel_10108/4265579779.py:16: RuntimeWarning: overflow encountered in double_scalars
  return 1/(1 + (math.e)**(-x))
